## Librerías

In [1]:
# Librerías de ciencia de datos 
import numpy as np
import pandas as pd
import sqlite3 as sql

# para análisis interactivo
from ipywidgets import interact, Output, widgets

# Librerías para preprocesamiento 
from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors 
import joblib

# Paquete para sistemas de recomendación surprise 
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise import Reader, Dataset

## Carga de datos

In [2]:
#### Conectar #######
conn=sql.connect('data/db_movies')
cur=conn.cursor()

In [3]:
#ver tablas
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratings_final',),
 ('movie_final',),
 ('full_ratings',),
 ('full_ratings_dum',),
 ('reco',)]

In [4]:
# Vamos a trabajar con el dataframe escalado y con dummies df_dum 
df_dum = pd.read_sql("SELECT * FROM full_ratings_dum", conn)
movies = pd.read_sql("SELECT * FROM movie_final", conn)

# Se selecciona un usuario para realizar las recomendaciones 
usuarios = pd.read_sql('SELECT distinct (userId) as user_id from ratings_final', conn)

# Seleccionamos un usuario para realizar el sistema de recomendación
user_id = 605
movies
ratings = pd.read_sql('SELECT * FROM ratings_final', conn)

## 3. Sistema de recomendación basado en contenido KNN Con base en todo lo visto por el usuario

### Top 10 de las películas más similares a las vistas por el usuario y con una que le haya dado una calificación mayor a 3.5.

In [8]:

# Función para recomendar películas
def recomendar(user_id, k=5):
    # Obtener los ratings del usuario seleccionado
    ratings = pd.read_sql('SELECT * FROM ratings_final WHERE userId=:user', conn, params={'user': user_id})
    
    # Filtrar solo películas con rating mayor a 4
    ratings = ratings[ratings['rating'] > 3.5]
    
    # Si no hay ratings, devolver mensaje
    if ratings.empty:
        return f"El usuario {user_id} no tiene películas calificadas con un rating mayor a 3.5."

    # Convertir ratings del usuario a array
    ratings_array = ratings['movieId'].to_numpy()
    
    # Agregar la columna de movieId y título al dataframe df_dum para filtrar y mostrar nombres
    df_dum[['movieId', 'title']] = movies[['movieId', 'title']]
    
    # Filtrar películas calificadas por el usuario
    movies_r = df_dum[df_dum['movieId'].isin(ratings_array)]
    
    # Eliminar columnas movieId y title
    movies_r = movies_r.drop(columns=['movieId', 'title'])
    movies_r["indice"] = 1  # Añadir índice para calcular el centroide
    
    # Calcular el centroide o perfil del usuario
    centroide = movies_r.groupby("indice").mean()
    
    # Filtrar películas no vistas por el usuario
    movies_ns = df_dum[~df_dum['movieId'].isin(ratings_array)]
    
    # Eliminar columnas movieId y title
    movies_ns = movies_ns.drop(columns=['movieId', 'title'])
    
    # Entrenar el modelo de K-Vecinos
    model = neighbors.NearestNeighbors(n_neighbors=k, metric='cosine')
    model.fit(movies_ns)
    
    # Obtener las películas más cercanas (recomendadas)
    dist, idlist = model.kneighbors(centroide)
    
    # Comprobar si hay suficientes vecinos cercanos
    if len(idlist[0]) < k:
        return f"No hay suficientes vecinos cercanos para el usuario {user_id}."

    # Obtener los títulos de las películas recomendadas
    ids = idlist[0]  # Desanidar el array de índices
    recomend_m = movies.loc[ids][['title']]
    
    return recomend_m

# Función para la interfaz interactiva
def interfaz_recomendaciones(user_id):
    recomendaciones_resultado = recomendar(user_id)
    print(f"Recomendaciones para el usuario {user_id}:")
    print(recomendaciones_resultado)

# Crear un widget interactivo para que el usuario seleccione el user_id
interact(interfaz_recomendaciones,
         user_id=widgets.IntSlider(min=1, max=1000, step=1, value=605, description='User ID:'))


interactive(children=(IntSlider(value=605, description='User ID:', max=1000, min=1), Output()), _dom_classes=(…

<function __main__.interfaz_recomendaciones(user_id)>

### 3.1. Top 5 de las películas más similares a las vistas por el usuario según el género

In [6]:
# Limpiar la columna 'genres': eliminar espacios extra y caracteres no deseados
movies['genres'] = movies['genres'].str.strip()

# Crear un widget Output para controlar las salidas
output = Output()

In [7]:


# Función para recomendar películas por género seleccionado y mostrar el rating promedio
def recomendar_peliculas_por_genero(genero_seleccionado, user_id, conn, top_n=5):
    # Se seleccionan los ratings del usuario establecido
    ratings_user = pd.read_sql('SELECT * from ratings_final where userId=:user', conn, params={'user': user_id})

    # Obtener el array de movieIds vistos por el usuario
    ratings_array = ratings_user['movieId'].to_numpy()

    # Filtrar películas vistas por el usuario
    peliculas_vistas = movies[movies['movieId'].isin(ratings_array)]

    # Expandir los géneros de las películas vistas por el usuario
    peliculas_vistas.loc[:, 'genres'] = peliculas_vistas['genres'].str.split('|')
    
    # Descomponer géneros en filas individuales (cada fila será una película-genre)
    peliculas_con_generos = peliculas_vistas.explode('genres')

    # Filtrar películas no vistas del género seleccionado
    peliculas_no_vistas = movies[~movies['movieId'].isin(ratings_array)]
    
    # Convertir el género seleccionado a string y eliminar espacios extra
    genero_seleccionado_str = str(genero_seleccionado).strip()

    # Filtrar las películas recomendadas por el género seleccionado
    peliculas_recomendadas = peliculas_no_vistas[peliculas_no_vistas['genres'].str.contains(genero_seleccionado_str, regex=False)]

    # Calcular el rating promedio para cada película
    rating_promedio = ratings.groupby('movieId')['rating'].mean().reset_index()

    # Unir la tabla de películas recomendadas con su rating promedio
    peliculas_recomendadas = peliculas_recomendadas.merge(rating_promedio, on='movieId', how='left')

    # Seleccionar las primeras top_n películas recomendadas y redondear el rating a 2 decimales
    peliculas_recomendadas['rating'] = peliculas_recomendadas['rating'].round(2)

    # Seleccionar las columnas 'title' y 'rating' para mostrar
    top_recomendadas = peliculas_recomendadas[['title', 'rating']].head(top_n)

    return top_recomendadas

# Obtener todos los géneros únicos de la columna 'genres'
todos_los_generos = movies['genres'].str.split('|').explode().unique()

# Crear la función interactiva con salida controlada


@interact
def mostrar_recomendaciones(genero=todos_los_generos):
    output.clear_output()  # Limpiar la salida anterior
    recomendaciones = recomendar_peliculas_por_genero(genero, user_id, conn)
    with output:
        print(f"Recomendaciones para el género '{genero}':")
        print(recomendaciones)

# Mostrar el widget de salida
display(output)


interactive(children=(Dropdown(description='genero', options=('Adventure', 'Animation', 'Children', 'Comedy', …

Output()

## 4. Sistema de recomendación filtro colaborativo

## Top 10 de las películas con mayor calificación predicha

In [10]:
#Indicar el id paraa el cual se requiere hacer la recomendación
user_id = 608

# Obtener datos; se filtran los mayores a 0
rat = pd.read_sql('SELECT * FROM ratings_final WHERE rating > 0', conn)

# Definir escala
reader = Reader(rating_scale=(1, 5))  # Corregí el nombre de la variable 'Reader' por 'reader'

# Leer datos con surprise
data = Dataset.load_from_df(rat[['userId', 'movieId', 'rating']], reader)

# Modelos 
models = [KNNBasic(), KNNWithMeans(), KNNWithZScore(), KNNBaseline()] 
results = {}

# Probar modelos
for model in models:
    CV_scores = cross_validate(model, data, measures=["MAE", "RMSE"], cv=5, n_jobs=-1)
    
    result = pd.DataFrame.from_dict(CV_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result

performance_df = pd.DataFrame.from_dict(results).T
performance_df.sort_values(by='RMSE')

# Se seleccionó el KNNBasic porque tiene mejores métricas en "MAE Y RMSE"

# Definir la grilla de parámetros para la búsqueda de hiperparámetros
param_grid = {
    'sim_options': {
        'name': ['msd', 'cosine'],
        'min_support': [5],
        'user_based': [False, True]
    }
}

gridsearchKNNBasic = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=2, n_jobs=-1)

gridsearchKNNBasic.fit(data)

gridsearchKNNBasic.best_params["rmse"]
gridsearchKNNBasic.best_score["rmse"]
gs_model = gridsearchKNNBasic.best_estimator['rmse']

# Entrenar con todos los datos, y realizar predicciones con el modelo afinado
trainset = data.build_full_trainset()
model1 = gs_model.fit(trainset)
predset = trainset.build_anti_testset()
len(predset)
predictions = gs_model.test(predset)

# Crear base; películas no vistas por usuario + calificación predicha 
predictions_df = pd.DataFrame(predictions) 
predictions_df.shape
predictions_df.head()
predictions_df['r_ui'].unique()
predictions_df.sort_values(by='est', ascending=False)

# Función para recomendar 11 películas a un usuario específico
def recomendaciones(user_id, n_recomend=11):
    predictions_userID = predictions_df[predictions_df['uid'] == user_id].\
                    sort_values(by="est", ascending=False).head(n_recomend)

    recomendados = predictions_userID[['iid', 'est']]  # Mantener 'iid' y 'est'
    recomendados.to_sql('reco', conn, if_exists="replace")

    # Cambié 'movies_1' por 'movies_sel' según tus tablas
    # Incluimos también la calificación estimada en la consulta
    mov = pd.read_sql('''SELECT b.title, a.est AS estimated_rating 
                         FROM reco a 
                         LEFT JOIN movie_final b ON a.iid = b.movieId''', conn)
    mov['estimated_rating'] = mov['estimated_rating'].round(2)
    
    return mov

# Obtener y mostrar las 11 películas recomendadas al usuario 50



Computing the msd similarity matrix...
Done computing similarity matrix.


In [11]:
n_recomend = 10
peliculas = recomendaciones(user_id, n_recomend)
peliculas

,title,estimated_rating
0,Heat (1995),4.05
1,Sense and Sensibility (1995),3.94
2,Goldfinger (1964),3.92
3,Inglourious Basterds (2009),3.90
4,Cool Hand Luke (1967),3.90
5,"Departed, The (2006)",3.90
6,"Good, the Bad and the Ugly, The (Buono, il bru...",3.89
7,Inception (2010),3.89
8,Brazil (1985),3.88
9,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",3.87
